In [35]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict_booster, load_model, Metrics, predict_xgbm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
# process_train_data()
# process_test_data()

train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_reduced_features_v2_tuning_v3.pickle"

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

None

-----------Train----------

Metrics
AUC: 0.97
Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98    140413
           1       0.88      0.48      0.62      9585

    accuracy                           0.96    149998
   macro avg       0.92      0.74      0.80    149998
weighted avg       0.96      0.96      0.96    149998

-----------Test----------

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.68      0.33      0.45      9403

    accuracy                           0.95    150000
   macro avg       0.82      0.66      0.71    150000
weighted avg       0.94      0.95      0.94    150000



: 